In [1]:
import geopandas as gpd

## Get County Data

In [2]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

In [3]:
counties_gdf = counties_gdf.to_crs(9311)

## Open McDonalds Data

In [4]:
mcdonalds_gdf = gpd.read_file("data/mcdonalds_va_h3_5.gpkg")
mcdonalds_gdf = mcdonalds_gdf.to_crs(9311)

## Get Subway Data

In [6]:
subway_gdf = gpd.read_file("data/subways.gpkg")
subway_gdf = subway_gdf.to_crs(9311)

## Intersect Data

In [7]:
counties_mcdonalds_count_gdf = counties_gdf.join(
    gpd.sjoin(mcdonalds_gdf, counties_gdf)
    .groupby("index_right")
    .size()
    .rename("stores"),
    how="left",
)

In [8]:
counties_mcdonalds_count_gdf = counties_mcdonalds_count_gdf.fillna(0)
counties_mcdonalds_count_gdf["mcdonalds"] = counties_mcdonalds_count_gdf[
    "stores"
].astype(int)
counties_mcdonalds_count_gdf = counties_mcdonalds_count_gdf[
    ["GEOID", "geometry", "mcdonalds", "NAME"]
]

In [9]:
counties_subway_count_gdf = counties_gdf.join(
    gpd.sjoin(subway_gdf, counties_gdf).groupby("index_right").size().rename("stores"),
    how="left",
)

In [10]:
counties_subway_count_gdf = counties_subway_count_gdf.fillna(0)
counties_subway_count_gdf["subways"] = counties_subway_count_gdf["stores"].astype(int)
counties_subway_count_gdf = counties_subway_count_gdf[
    ["GEOID", "geometry", "subways", "NAME"]
]

#### Merge Data now

In [11]:
mcd_subway_gdf = counties_mcdonalds_count_gdf.merge(
    counties_subway_count_gdf[["GEOID", "subways"]],
    on="GEOID",
    how="left",
)

In [12]:
def mcdonalds_subway_classification(row) -> str:
    subway = row["subways"]
    md = row["mcdonalds"]
    if subway == 0 and md == 0:
        return "no stores"
    if subway == md:
        return "same amount"
    if md > subway:
        return "more mcdonald's"
    if md < subway:
        return "more subways"

In [13]:
mcd_subway_gdf["md_or_subway"] = mcd_subway_gdf.apply(
    lambda row: mcdonalds_subway_classification(row), axis=1
)

In [14]:
mcd_subway_gdf = mcd_subway_gdf.to_crs(9311)

In [15]:
mcd_subway_gdf.to_file("data/mcd_subway_gdf.gpkg")